<a href="https://colab.research.google.com/github/sheelapravalika/AI_CS_sheelapravalika13gmail/blob/main/SLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U datasets setfit scikit-learn pandas matplotlib seaborn


In [ ]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("sonnh-tech1/cic-ids-2017", "binary")
df = dataset["train"].to_pandas()

# Reduce the dataset size for faster SLM fine-tuning
df = df.sample(n=5000, random_state=42)

# Encode labels
df = df[['FlowDuration', 'TotalFwdPackets', 'TotalBackwardPackets', 'Label']]  # Select minimal features
df['Label'] = df['Label'].astype('category').cat.codes  # BENIGN=0, ATTACK=1

df.head()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00004.parquet:   0%|          | 0.00/92.8M [00:00<?, ?B/s]

train-00001-of-00004.parquet:   0%|          | 0.00/107M [00:00<?, ?B/s]

train-00002-of-00004.parquet:   0%|          | 0.00/108M [00:00<?, ?B/s]

train-00003-of-00004.parquet:   0%|          | 0.00/107M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2522362 [00:00<?, ? examples/s]

,FlowDuration,TotalFwdPackets,TotalBackwardPackets,Label
844940,5352004,3,1,0
918226,72577,1,1,0
1649965,293661,2,0,0
589768,116069073,25,21,0
1166898,30243318,8,8,0


In [ ]:
# Convert features to string for transformer input
df['text'] = df[['FlowDuration', 'TotalFwdPackets', 'TotalBackwardPackets']].astype(str).agg(' '.join, axis=1)
df = df[['text', 'Label']]
df.head()

,text,Label
844940,5352004 3 1,0
918226,72577 1 1,0
1649965,293661 2 0,0
589768,116069073 25 21,0
1166898,30243318 8 8,0


In [ ]:
from setfit import SetFitModel, SetFitTrainer
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Count the occurrences of each label
label_counts = df['Label'].value_counts()

# Identify labels with only one occurrence
single_occurrence_labels = label_counts[label_counts == 1].index

# Filter out rows with single occurrence labels
filtered_df = df[~df['Label'].isin(single_occurrence_labels)]


train_df, test_df = train_test_split(filtered_df, test_size=0.2, stratify=filtered_df['Label'], random_state=42)

In [ ]:
from datasets import Dataset
from setfit import SetFitModel, SetFitTrainer

In [ ]:
train_df["Label"] = train_df["Label"].apply(int)
test_df["Label"] = test_df["Label"].apply(int)

In [ ]:
train_dataset = Dataset.from_dict({
    "text": train_df["text"].tolist(),
    "label": train_df["Label"].tolist()
})

test_dataset = Dataset.from_dict({
    "text": test_df["text"].tolist(),
    "label": test_df["Label"].tolist()
})



In [ ]:
# STEP 9: Load small language model from Sentence Transformers
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-MiniLM-L6-v2")

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [ ]:
print(train_dataset.column_names)

['text', 'label']


In [ ]:
pip install -U setfit

In [ ]:
# Initialize Trainer
from setfit import SetFitTrainer

trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    batch_size=16,
    num_iterations=10,
    #metric_keys='accuracy', # Added required metric_keys argument
)

/tmp/ipython-input-28-2891848183.py:4: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(


Map:   0%|          | 0/3999 [00:00<?, ? examples/s]

In [ ]:

# Train
trainer.train()


***** Running training *****
  Num unique pairs = 79980
  Batch size = 16
  Num epochs = 1


Step,Training Loss
1,0.046400
50,0.063200
100,0.050700
150,0.048000
200,0.050100
250,0.053200
300,0.061300
350,0.057300
400,0.064900
450,0.067200


TypeError: Wrong key type: '2006' of type '<class 'numpy.int64'>'. Expected one of int, slice, range, str or Iterable.